# Delivery Time

# 1. Preparation

In [1]:
import pandas as pd
import datetime

# 2. Import Data

In [2]:
data = {'Order': ['2020/1/1 14:00', '2020/1/1 14:00', '2020/1/1 14:00', 
                  '2020/1/1 23:00', '2020/1/1 23:00', '2020/1/1 23:00', '2020/1/1 9:00'],
        'Delivery': ['2020/1/8 11:00', '2020/1/8 7:00', '2020/1/2 11:00', 
                     '2020/1/8 11:00', '2020/1/8 7:00', '2020/1/2 11:00', '2020/1/1 18:00']}  

In [3]:
df = pd.DataFrame(data)

In [4]:
df

,Order,Delivery
0,2020/1/1 14:00,2020/1/8 11:00
1,2020/1/1 14:00,2020/1/8 7:00
2,2020/1/1 14:00,2020/1/2 11:00
3,2020/1/1 23:00,2020/1/8 11:00
4,2020/1/1 23:00,2020/1/8 7:00
5,2020/1/1 23:00,2020/1/2 11:00
6,2020/1/1 9:00,2020/1/1 18:00


In [5]:
df.dtypes

Order       object
Delivery    object
dtype: object

In [6]:
df['Order'] = pd.to_datetime(df['Order'])
df['Delivery'] = pd.to_datetime(df['Delivery'])

In [7]:
df.dtypes

Order       datetime64[ns]
Delivery    datetime64[ns]
dtype: object

# 3. Calculation

## 3.1. Time Addition

In [8]:
def time_addition(order, delivery):
    order_date, order_time, delivery_date, delivery_time = order.date(), order.time(), delivery.date(), delivery.time()
    order_21 = datetime.datetime(order.year, order.month, order.day, 21, 0)
    delivery_8 = datetime.datetime(delivery.year, delivery.month, delivery.day, 8, 0) 
    if order_date != delivery_date:
        first_day_duration = max((order_21-order).total_seconds() / 3600, 0)
        last_day_duration = max((delivery-delivery_8).total_seconds() / 3600, 0)
        delivery_duration = (delivery_date-order_date).days-1
        total_duration = first_day_duration + last_day_duration + delivery_duration*13        
    elif order_date == delivery_date:
        total_duration = (delivery-order).total_seconds() / 3600
    return total_duration

## 3.2. Time Deduction

In [9]:
def time_deduction(order, delivery):
    order_date, order_time, delivery_date, delivery_time = order.date(), order.time(), delivery.date(), delivery.time()
    order_21 = datetime.datetime(order.year, order.month, order.day, 21, 0)
    delivery_8 = datetime.datetime(delivery.year, delivery.month, delivery.day, 8, 0)
    
    duration = (delivery-order).total_seconds() / 3600    
    deduction = (delivery_date-order_date).days 
    
    first_day_deduction = min((order_21-order).total_seconds() / 3600, 0)
    last_day_deduction = min((delivery-delivery_8).total_seconds() / 3600, 0)
    
    total_duration = duration - first_day_deduction - last_day_deduction - deduction*11
    return total_duration

In [10]:
def df_apply_delivery_time(row, is_addition):
    if is_addition:
        duration = time_addition(row['Order'], row['Delivery'])
    else: 
        duration = time_deduction(row['Order'], row['Delivery'])
    return duration

In [11]:
df['Delivery Time (Addition)'] = df.apply(df_apply_delivery_time, is_addition=True, axis=1)
df['Delivery Time (Deduction)'] = df.apply(df_apply_delivery_time, is_addition=False, axis=1)

In [12]:
df

,Order,Delivery,Delivery Time (Addition),Delivery Time (Deduction)
0,2020-01-01 14:00:00,2020-01-08 11:00:00,88.0,88.0
1,2020-01-01 14:00:00,2020-01-08 07:00:00,85.0,85.0
2,2020-01-01 14:00:00,2020-01-02 11:00:00,10.0,10.0
3,2020-01-01 23:00:00,2020-01-08 11:00:00,81.0,81.0
4,2020-01-01 23:00:00,2020-01-08 07:00:00,78.0,78.0
5,2020-01-01 23:00:00,2020-01-02 11:00:00,3.0,3.0
6,2020-01-01 09:00:00,2020-01-01 18:00:00,9.0,9.0
